In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import pandas as pd
import os, sys
sys.path.append(os.path.join(os.getcwd(), 'src'))
from dataset_analyze import *
from database_op import DataBaseOP
from tqdm.notebook import tqdm
from functools import reduce

# 读取所有数据集。数据集较大，有3GB，请保证有足够的内存空间

In [2]:
df = pd.read_csv('./dataset/UserBehavior.csv', header=None)
df_temp = df.iloc[: 10, :]
df.columns = ['用户ID', '商品ID', '商品类目ID', '行为类型', '时间戳']
df

,用户ID,商品ID,商品类目ID,行为类型,时间戳
0,1,2268318,2520377,pv,1511544070
1,1,2333346,2520771,pv,1511561733
2,1,2576651,149192,pv,1511572885
3,1,3830808,4181361,pv,1511593493
4,1,4365585,2520377,pv,1511596146
...,...,...,...,...,...
100150802,999999,4797808,11120,pv,1512293403
100150803,999999,4613472,4602841,pv,1512293766
100150804,999999,3647364,2304296,pv,1512293792
100150805,999999,1903801,2304296,pv,1512293827


# 数据处理1：统计一个用户每一种行为数量 为top n的商品类目ID

In [59]:
# connect database
dbop = DataBaseOP()
dbop.select_table('bigdata_userbehaviortopn')

# for table bigdata_userbehaviortopn, attr is fixed
attr = 'id, user_id, pv, buy, cart, fav'
top_n = 5

# maybe the statistic results is empty
def empty2zero(Series):
    if Series.empty:
        return 0
    return reduce(lambda x, y: f'{x}\t{y}', Series.items())

# for each loop we can only handle single user
# 
# for too many users, so we take user id [1, 1000)
for i in tqdm(range(1, 1000)):
    item = single_user_behavior_statistics(df, i, top_n)
    
    pv = empty2zero(item[i]['pv'])
    buy = empty2zero(item[i]['buy'])
    cart = empty2zero(item[i]['cart'])
    fav = empty2zero(item[i]['fav'])
    val = f'{i}, "{i}", "{pv}", "{buy}", "{cart}", "{fav}"'
    # insert
    dbop.insert(attr, val)
    
dbop.close()

# 数据处理2：统计各类商品在四个行为中的排名

In [39]:
pcis = productsClassID_statistics(df)